# FairLens – Final Model Training Notebook

XGBoost + Calibration + Binary Threshold (0.55)

In [ ]:

!pip install xgboost shap --quiet

import numpy as np
import pandas as pd
import shap
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier


## Train/Test Split

In [ ]:

feature_columns = [
    "avg_monthly_inflow",
    "inflow_volatility",
    "avg_monthly_outflow",
    "min_balance_30d",
    "neg_balance_days_30d",
    "purchase_to_inflow_ratio",
    "total_burden_ratio",
    "buffer_ratio",
    "stress_index"
]

X = df[feature_columns]
y = df["default_next_cycle"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


## Train XGBoost Model

In [ ]:

scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])

model = XGBClassifier(
    n_estimators=600,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.85,
    scale_pos_weight=scale_pos_weight,
    eval_metric="auc",
    random_state=42
)

model.fit(X_train, y_train)


## Calibration & Evaluation

In [ ]:

calibrated_model = CalibratedClassifierCV(model, method="isotonic", cv=3)
calibrated_model.fit(X_train, y_train)

y_proba = calibrated_model.predict_proba(X_test)[:,1]

print("ROC-AUC:", roc_auc_score(y_test, y_proba))

THRESHOLD = 0.55
y_pred = (y_proba >= THRESHOLD).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


## SHAP Explainability

In [ ]:

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test)


## Export Model

In [ ]:

joblib.dump(calibrated_model, "bnpl_cashflow_model.pkl")

import json
metadata = {
    "threshold": THRESHOLD,
    "feature_columns": feature_columns
}

with open("model_metadata.json","w") as f:
    json.dump(metadata,f)

print("Model and metadata exported.")
